In [0]:
scope = 'cxkeyvault'
sas_token_key = 'cxdatalake'
storage_account = 'customeranalysisdatalake'
container = 'raw'
container2 = 'silver'

sas_token = dbutils.secrets.get(scope=scope, key=sas_token_key)



import os
from datetime import datetime
from delta.tables import DeltaTable
from pyspark.sql.functions import col, when

In [0]:
spark.conf.set(f"fs.azure.account.auth.type.{storage_account}.dfs.core.windows.net", "SAS")
spark.conf.set(f"fs.azure.sas.token.provider.type.{storage_account}.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.sas.FixedSASTokenProvider")
spark.conf.set(f"fs.azure.sas.fixed.token.{storage_account}.dfs.core.windows.net", sas_token)


In [0]:
#all paths

accounts_deltatable_path = (f"abfss://{container2}@{storage_account}.dfs.core.windows.net/accounts/")
customers_deltatable_path = (f"abfss://{container2}@{storage_account}.dfs.core.windows.net/customers/")
loan_payments_deltatable_path = (f"abfss://{container2}@{storage_account}.dfs.core.windows.net/loan payments/")
loans_deltatable_path = (f"abfss://{container2}@{storage_account}.dfs.core.windows.net/loans/")
transactions_deltatable_path = (f"abfss://{container2}@{storage_account}.dfs.core.windows.net/transactions/")

accounts_raw_path = (f"abfss://{container}@{storage_account}.dfs.core.windows.net/accounts/")
customers_raw_path = (f"abfss://{container}@{storage_account}.dfs.core.windows.net/customers/")
loan_payments_raw_path = (f"abfss://{container}@{storage_account}.dfs.core.windows.net/loan payments/")
loans_raw_path = (f"abfss://{container}@{storage_account}.dfs.core.windows.net/loans/")
transactions_raw_path = (f"abfss://{container}@{storage_account}.dfs.core.windows.net/transactions/")



In [0]:


# #Accounts delta lake empty table

# from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DecimalType
# schema = StructType([ StructField("account_id", IntegerType(), True),
#                      StructField("customer_id", IntegerType(), True), 
#                      StructField("account_type", StringType(), True), 
#                      StructField("balance", DecimalType(10,2), True)
#                      ])

# accounts_deltatable_path = (f"abfss://{container2}@{storage_account}.dfs.core.windows.net/accounts/")

# # Create an empty DataFrame with the schema 
# empty_df = spark.createDataFrame([], schema)

# empty_df.write.format("delta").save(accounts_deltatable_path)


In [0]:

# #customers delta lake empty table

# from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DecimalType
# schema = StructType([
#                     StructField("customer_id", IntegerType(), True), 
#                     StructField("first_name", StringType(), True),
#                     StructField("last_name", StringType(), True),
#                     StructField("address", StringType(), True),
#                     StructField("city", StringType(), True),
#                     StructField("state", StringType(), True),
#                     StructField("zip", StringType(), True),
#                      ])

# customers_deltatable_path = (f"abfss://{container2}@{storage_account}.dfs.core.windows.net/customers/")

# # Create an empty DataFrame with the schema 
# empty_df = spark.createDataFrame([], schema)

# empty_df.write.format("delta").save(customers_deltatable_path)

In [0]:
# #loan payments delta lake empty table

# from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DecimalType, DateType
# schema = StructType([
#                     StructField("payment_id", IntegerType(), True),
#                     StructField("loan_id", IntegerType(), True),
#                     StructField("payment_date", DateType(), True),
#                     StructField("payment_amount", IntegerType(), True),
#                      ])

# loan_payments_deltatable_path = (f"abfss://{container2}@{storage_account}.dfs.core.windows.net/loan payments/")

# # Create an empty DataFrame with the schema 
# empty_df = spark.createDataFrame([], schema)

# empty_df.write.format("delta").save(loan_payments_deltatable_path)

In [0]:
# #loans delta lake empty table

# from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DecimalType, DateType
# schema = StructType([
#                     StructField("loan_id", IntegerType(), True),
#                     StructField("customer_id", IntegerType(), True),
#                     StructField("loan_amount", DecimalType(10,2), True),
#                     StructField("interest_rate", DecimalType(4,2), True),
#                     StructField("loan_term", IntegerType(), True),
#                      ])

# loans_deltatable_path = (f"abfss://{container2}@{storage_account}.dfs.core.windows.net/loans/")

# # Create an empty DataFrame with the schema 
# empty_df = spark.createDataFrame([], schema)

# empty_df.write.format("delta").save(loans_deltatable_path)

In [0]:
# #transactions delta lake empty table

# from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DecimalType, DateType
# schema = StructType([
#                     StructField("transaction_id", IntegerType(), True),
#                     StructField("account_id", IntegerType(), True),
#                     StructField("transaction_date", DateType(), True),
#                     StructField("transaction_amount", DecimalType(10,2), True),
#                     StructField("transaction_type", StringType(), True),
#                      ])

# transactions_deltatable_path = (f"abfss://{container2}@{storage_account}.dfs.core.windows.net/transactions/")

# # Create an empty DataFrame with the schema 
# empty_df = spark.createDataFrame([], schema)

# empty_df.write.format("delta").save(transactions_deltatable_path)

In [0]:
def get_last_commit_timestamp(deltatable_path):
  # Load the Delta table
  delta_table = DeltaTable.forPath(spark, deltatable_path)

  # Get the history of the Delta table
  history_df = delta_table.history()

  # Get the timestamp of the last commit
  last_commit_timestamp = history_df.orderBy(history_df['version'].desc()).select('timestamp').first()['timestamp']
  return last_commit_timestamp


In [0]:
def list_and_filter_files(last_commit_datetime, raw_path,expected_columns):
  new_files = []
  files = dbutils.fs.ls(raw_path)
  for file in files:
    if (datetime.fromtimestamp(file.modificationTime/1000))> last_commit_datetime:
      df = spark.read.option("header", "true").csv(file.path)
      if set(df.columns) == set(expected_columns):
        new_files.append(file.path)
  return new_files

In [0]:
#accounts csv files transfer from raw to silver parquet

expected_columns = (
    spark.read.format("delta")
    .load(accounts_deltatable_path)
    .schema.names
)
last_commit_datetime = get_last_commit_timestamp(accounts_deltatable_path)
filtered_files = list_and_filter_files(last_commit_datetime,accounts_raw_path, expected_columns)

if filtered_files:
  accounts_df = spark.read.option("header", "true").csv(filtered_files)

  # Remove duplicates 
  accounts_df = accounts_df.dropDuplicates()

  #df = df.select([when(col(c) == "", None).otherwise(col(c)).alias(c) for c in df.columns])

  accounts_df = accounts_df.withColumn("account_id", col("account_id").cast("integer"))
  accounts_df = accounts_df.withColumn("customer_id", col("customer_id").cast("integer"))
  accounts_df = accounts_df.withColumn("account_type", col("account_type").cast("string"))
  accounts_df = accounts_df.withColumn("balance", col("balance").cast("decimal(10,2)"))

  accounts_df.write.format("delta").mode("append").save(accounts_deltatable_path)

else:
  print("No new files found in the accounts")



No new files found in the accounts


In [0]:
#customer csv files transfer from raw to silver parquet

expected_columns = (
    spark.read.format("delta")
    .load(customers_deltatable_path)
    .schema.names
)
last_commit_datetime = get_last_commit_timestamp(customers_deltatable_path)
filtered_files = list_and_filter_files(last_commit_datetime,customers_raw_path, expected_columns)

if filtered_files:
  customer_df = spark.read.option("header", "true").csv(filtered_files)

  # Remove duplicates 
  customer_df = customer_df.dropDuplicates()

  #df = df.select([when(col(c) == "", None).otherwise(col(c)).alias(c) for c in df.columns])

  customer_df = customer_df.withColumn("customer_id", col("customer_id").cast("integer"))
  customer_df = customer_df.withColumn("first_name", col("first_name").cast("string"))
  customer_df = customer_df.withColumn("last_name", col("last_name").cast("string"))
  customer_df = customer_df.withColumn("address", col("address").cast("string"))
  customer_df = customer_df.withColumn("city", col("city").cast("string"))
  customer_df = customer_df.withColumn("state", col("state").cast("string"))
  customer_df = customer_df.withColumn("zip", col("zip").cast("string"))

  customer_df.write.format("delta").mode("append").save(customers_deltatable_path)

else:
  print("No new files found in the customers")



No new files found in the accounts


In [0]:
#loan payments csv files transfer from raw to silver parquet

expected_columns = (
    spark.read.format("delta")
    .load(loan_payments_deltatable_path)
    .schema.names
)
last_commit_datetime = get_last_commit_timestamp(loan_payments_deltatable_path)
filtered_files = list_and_filter_files(last_commit_datetime,loan_payments_raw_path, expected_columns)

if filtered_files:
  loan_payments_df = spark.read.option("header", "true").csv(filtered_files)

  # Remove duplicates 
  loan_payments_df = loan_payments_df.dropDuplicates()

  #df = df.select([when(col(c) == "", None).otherwise(col(c)).alias(c) for c in df.columns])

  loan_payments_df = loan_payments_df.withColumn("payment_id", col("payment_id").cast("integer"))
  loan_payments_df = loan_payments_df.withColumn("loan_id", col("loan_id").cast("integer"))
  loan_payments_df = loan_payments_df.withColumn("payment_date", col("payment_date").cast("date"))
  loan_payments_df = loan_payments_df.withColumn("payment_amount", col("payment_amount").cast("integer"))

  loan_payments_df.write.format("delta").mode("append").save(loan_payments_deltatable_path)

else:
  print("No new files found in the loan payments")



No new files found in the loan payments


In [0]:
#loans csv files transfer from raw to silver parquet

expected_columns = (
    spark.read.format("delta")
    .load(loans_deltatable_path)
    .schema.names
)
last_commit_datetime = get_last_commit_timestamp(loans_deltatable_path)
filtered_files = list_and_filter_files(last_commit_datetime,loans_raw_path, expected_columns)

if filtered_files:
  loans_df = spark.read.option("header", "true").csv(filtered_files)

  # Remove duplicates 
  loans_df = loans_df.dropDuplicates()

  #df = df.select([when(col(c) == "", None).otherwise(col(c)).alias(c) for c in df.columns])

  loans_df = loans_df.withColumn("loan_id", col("loan_id").cast("integer"))
  loans_df = loans_df.withColumn("customer_id", col("customer_id").cast("integer"))
  loans_df = loans_df.withColumn("loan_amount", col("loan_amount").cast("decimal(10, 2)"))
  loans_df = loans_df.withColumn("interest_rate", col("interest_rate").cast("decimal(4, 2)"))
  loans_df = loans_df.withColumn("loan_term", col("loan_term").cast("integer"))


  loans_df.write.format("delta").mode("append").save(loans_deltatable_path)

else:
  print("No new files found in the loans")



No new files found in the loans


In [0]:
#transactions csv files transfer from raw to silver parquet

expected_columns = (
    spark.read.format("delta")
    .load(transactions_deltatable_path)
    .schema.names
)
last_commit_datetime = get_last_commit_timestamp(transactions_deltatable_path)
filtered_files = list_and_filter_files(last_commit_datetime,transactions_raw_path, expected_columns)

if filtered_files:
  transactions_df = spark.read.option("header", "true").csv(filtered_files)

  # Remove duplicates 
  transactions_df = transactions_df.dropDuplicates()

  #df = df.select([when(col(c) == "", None).otherwise(col(c)).alias(c) for c in df.columns])

  transactions_df = transactions_df.withColumn("transaction_id", col("transaction_id").cast("integer"))
  transactions_df = transactions_df.withColumn("account_id", col("account_id").cast("integer"))
  transactions_df = transactions_df.withColumn("transaction_date", col("transaction_date").cast("date"))
  transactions_df = transactions_df.withColumn("transaction_amount", col("transaction_amount").cast("decimal(10, 2)"))
  transactions_df = transactions_df.withColumn("transaction_type", col("transaction_type").cast("string"))


  transactions_df.write.format("delta").mode("append").save(transactions_deltatable_path)

else:
  print("No new files found in the loans")



No new files found in the loans
